# 🧹 บทที่ 4: การชำระล้างข้อมูล (Data Cleaning & Sanitization)
---

ในงานระดับมือโปร เราจะเชื่อใจข้อมูลที่โหลดมา 100% ไม่ได้ครับ บางที `MSISDN` หายไป หรือ `RC_RATE` กลายเป็นค่าว่าง (Null) ถ้าเราเอาไปคำนวณต่อ โปรแกรมจะพังทันที บทนี้เราจะเรียนรู้วิธี "ดักจับ" และ "แก้ไข" ข้อมูลพวกนี้ครับ

| คำศัพท์ (Technical Term) | คำอ่านไทย (Pronunciation) | ความหมาย (Translation/Definition) |
| :--- | :--- | :--- |
| **Sanitization** | แซ-นิ-ไท-เซ-ชัน | การทำความสะอาดข้อมูลให้พร้อมใช้งาน |
| **Missing Value / NaN** | มิส-ซิง แว-ลู / แนน | ข้อมูลที่หายไป หรือค่าที่ว่างเปล่า (Not a Number) |
| **Imputation** | อิม-พิว-เท-ชัน | การเติมค่าที่หายไปด้วยค่าที่เรากำหนด (เช่น เติม 0 แทนค่าว่าง) |
| **Drop** | ดร็อป | การทิ้งข้อมูลที่ไม่ต้องการ หรือแถวที่ไม่สมบูรณ์ |

### 🔍 สิ่งที่ป๋าต้องทบทวน (Review)
1. การใช้ `pd.ExcelFile` เพื่อเปิดไฟล์
2. การใช้ `dtype` เพื่อคุมประเภทข้อมูล
3. การใช้ `info()` เพื่อส่องดูว่าคอลัมน์ไหนมีค่าว่าง (Non-Null Count)

In [1]:
import pandas as pd
import numpy as np

# สร้างข้อมูลที่มีความ "สกปรก" เล็กน้อย
dirty_data = {
    'MSISDN': ['0960000001', np.nan, '0960000003', '0960000004', '0960000005'], # มีค่า NaN (ว่าง)
    'CUST_NAME': ['A Corp', 'B Co.', 'C Shop', np.nan, 'E Ltd.'],               # มีชื่อว่าง
    'RC_RATE': [599, 899, np.nan, 449, 1200],                                    # มีราคาวาง
    'CONTRACT_END_DT': ['2025-12-31', '2026-05-20', '2026-12-31', np.nan, '2027-01-01'] # มีวันที่ว่าง
}

df_dirty = pd.DataFrame(dirty_data)
df_dirty.to_excel('../Lesson_2/isp_dirty_data.xlsx', index=False)
print("ป๋าครับ! ไฟล์ข้อมูลแบบ 'สกปรก' พร้อมให้ป๋าซักฟอกแล้วครับ")

ป๋าครับ! ไฟล์ข้อมูลแบบ 'สกปรก' พร้อมให้ป๋าซักฟอกแล้วครับ


## 🧼 ขั้นตอนการซักฟอกข้อมูล (The Cleaning Process)
---

เมื่อป๋าโหลดข้อมูลมาแล้ว สิ่งแรกที่ต้องทำคือการตรวจดูว่า "รูรั่ว" อยู่ตรงไหนบ้าง โดยใช้คำสั่งกลุ่ม `isna` ครับ

| คำศัพท์ (Technical Term) | คำอ่านไทย (Pronunciation) | ความหมาย (Translation/Definition) |
| :--- | :--- | :--- |
| **isna() / isnull()** | อีส-น่า / อีส-นัล | คำสั่งเช็กว่า "ว่างใช่ไหม?" (ถ้าว่างจะเป็น True) |
| **fillna()** | ฟิล-น่า | คำสั่งสำหรับ "เติมคำในช่องว่าง" |
| **dropna()** | ดร็อป-น่า | คำสั่งสำหรับ "ลบทิ้งทั้งแถว" ถ้ามีค่าว่าง |

> **กลยุทธ์ของป๋า:** > 1. ถ้าข้อมูลสำคัญหาย (เช่น MSISDN) -> **ลบทิ้งทั้งแถว**
> 2. ถ้าข้อมูลตัวเลขหาย (เช่น ราคา) -> **เติม 0 หรือค่าเฉลี่ยแทน**

In [2]:
# 1. ทบทวนบทเก่า: โหลดไฟล์และเช็กสุขภาพข้อมูล
excel = pd.ExcelFile('../Lesson_2/isp_dirty_data.xlsx')
df = excel.parse(0, dtype={'MSISDN': str})

print("ตรวจสอบค่าว่างรายคอลัมน์:")
print(df.isna().sum()) # จะบอกว่าแต่ละหัวข้อมีค่าว่างกี่จุด

# 2. ป๋าตัดสินใจ: "ถ้าไม่มี MSISDN ป๋าไม่เอา!" (ลบแถวที่ MSISDN ว่าง)
df = df.dropna(subset=['MSISDN'])

# 3. ป๋าใจดี: "ถ้าราคาว่าง ให้ถือว่าเป็น 0 ไปก่อน"
df['RC_RATE'] = df['RC_RATE'].fillna(0)

# 4. ป๋าละเอียด: "ถ้าชื่อว่าง ให้ใส่ว่า 'Unknown Customer'"
df['CUST_NAME'] = df['CUST_NAME'].fillna('Unknown Customer')

# 5. ทบทวนบทเก่า: แปลงวันที่และจัดการค่าว่างของวันที่
df['CONTRACT_END_DT'] = pd.to_datetime(df['CONTRACT_END_DT'])
# ถ้าวันสิ้นสุดสัญญาว่าง ให้เติมวันไกลๆ ไว้ก่อน (เช่น ปี 2099)
df['CONTRACT_END_DT'] = df['CONTRACT_END_DT'].fillna(pd.Timestamp('2099-12-31'))

display(df)

ตรวจสอบค่าว่างรายคอลัมน์:
MSISDN             1
CUST_NAME          1
RC_RATE            1
CONTRACT_END_DT    1
dtype: int64


,MSISDN,CUST_NAME,RC_RATE,CONTRACT_END_DT
0,0960000001,A Corp,599.0,2025-12-31
2,0960000003,C Shop,0.0,2026-12-31
3,0960000004,Unknown Customer,449.0,2099-12-31
4,0960000005,E Ltd.,1200.0,2027-01-01
